In [1]:
from pyscf import dft, gto
from utils import unit, atom
from sTDA.sTDAopt import sTDA
from sTDA.UsTDAopt import UsTDA
from sTDA.XsTDAopt import XsTDA

# sTDA

In [2]:
mol = gto.M(
    atom=atom.ch2o_vacuum,  # unit is Angstrom  # in utils/atom.py, give lots of atom
    unit="A",
    basis = "cc-pvdz",
    spin=0,
    charge=0,
    verbose=4
)
mf = dft.RKS(mol)
mf.conv_tol = 1e-11  # set small to compare pyscf result with my result
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
xc = 'b3lyp'
mf.xc = xc
mf.conv_check = False  # check convergence will run another cycle making result change
mf.kernel()

# truncation Threshold unit is eV
stda = sTDA(mol, mf, singlet=True, nstates=10, cas=True, truncate=20, correct=False)
stda.info()
print("="*50)
stda.nstates = 12
stda.singlet = True  # True stand for singlet excited, Fasle stand for triplet excited
# TODO(WHB): make scsf threshold can be change
stda.truncate = 20  # eV, pscf threshold
stda.cas = True
stda.correct = False  # diagonal correct, https://doi.org/10.1063/1.4959605
stda.savedata = False  # set True will get "uvspec_data.csv" saving excited energy and corresponding oscillator strength
e_eV, os, rs, v, pscsf = stda.kernel()

System: uname_result(system='Linux', node='ubuntu', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 4
Python 3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0]
numpy 2.2.6  scipy 1.15.2  h5py 3.13.0
Date: Thu Jun 26 23:00:45 2025
PySCF version 2.8.0
PySCF path  /home/whb/Downloads/miniconda3/envs/TDDFT/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 16
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.526270000000   0.000000000000 AA    0.000000000000   0.994506167575   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.979180000000   1.091955000000   0.000000000000 AA    1.850382026652   2.06349589034

/home/whb/Downloads/miniconda3/envs/TDDFT/lib/python3.11/site-packages/pyscf/dft/libxc.py:512: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


tot grids = 47816
init E= -114.623380393791
  HOMO = -0.324701828648962  LUMO = -0.112033005948581
cycle= 1 E= -114.38578625568  delta_E= 0.238  |g|= 0.768  |ddm|= 1.83
  HOMO = -0.105630276145629  LUMO = 0.0261329114483787
cycle= 2 E= -113.850850435971  delta_E= 0.535  |g|= 1.77  |ddm|= 2.09
  HOMO = -0.257687387322565  LUMO = -0.035177209681839
cycle= 3 E= -114.5059260314  delta_E= -0.655  |g|= 0.0463  |ddm|= 1.61
  HOMO = -0.268478935764856  LUMO = -0.0467741542050979
cycle= 4 E= -114.506273632547  delta_E= -0.000348  |g|= 0.034  |ddm|= 0.0723
  HOMO = -0.264092223748706  LUMO = -0.0395959573383574
cycle= 5 E= -114.50655158283  delta_E= -0.000278  |g|= 0.00705  |ddm|= 0.0274
  HOMO = -0.264971941700297  LUMO = -0.04008227221971
cycle= 6 E= -114.506562041579  delta_E= -1.05e-05  |g|= 0.00147  |ddm|= 0.00625
  HOMO = -0.265131587847431  LUMO = -0.0401462308511737
cycle= 7 E= -114.50656246549  delta_E= -4.24e-07  |g|= 0.000116  |ddm|= 0.00135
  HOMO = -0.265138976148809  LUMO = -0.0401

# UsTDA

In [4]:
mol = gto.M(
    atom=atom.ch2o_vacuum,
    # atom=atom.ch2o_cyclohexane,
    unit="A",
    basis='cc-pvdz',
    spin=1,
    charge=1,
    verbose=4
)

# # ground state calculation add solvents
# # add solvents
# mf = dft.UKS(mol).SMD()
# # in https://gaussian.com/scrf/ solvents entry, give different eps for different solvents
# # mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# # mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# # mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃

mf = dft.UKS(mol)  # if include solvent effect, remember comment this line
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
xc = 'b3lyp'
mf.xc = xc
mf.conv_check = False
mf.kernel()

ustda = UsTDA(mol, mf)
ustda.info()
print("="*50)
ustda.paramtype = 'os'  # os or cs, os is stand for open shell, cs is for close shell
ustda.nstates = 12
ustda.truncate = 20.0
ustda.cas = True
ustda.savedata = False
e_eV, os, rs, v, pscsf = ustda.kernel()

System: uname_result(system='Linux', node='ubuntu', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 4
Python 3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0]
numpy 2.2.6  scipy 1.15.2  h5py 3.13.0
Date: Thu Jun 26 23:04:34 2025
PySCF version 2.8.0
PySCF path  /home/whb/Downloads/miniconda3/envs/TDDFT/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.526270000000   0.000000000000 AA    0.000000000000   0.994506167575   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.979180000000   1.091955000000   0.000000000000 AA    1.850382026652   2.06349589034

# XsTDA

In [6]:
mol = gto.M(
    # atom=atom.ch2o_vacuum,
    atom=atom.ch2o_cyclohexane,
    unit="A",
    basis='cc-pvdz',
    spin=1,
    charge=1,
    verbose=4
)

# add solvents
mf = dft.ROKS(mol).SMD()
# in https://gaussian.com/scrf/ solvents entry, give different eps for different solvents
mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃

# mf = dft.ROKS(mol)
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
mf.xc = 'b3lyp'
mf.conv_check = False
mf.kernel()

xstda = XsTDA(mol, mf)
xstda.info()
print("="*50)
xstda.paramtype = 'os'
xstda.nstates = 12
xstda.cas = True
xstda.truncate = 20
xstda.correct = False
xstda.savedata = False
e_eV, os, rs, v, pscsf = xstda.kernel()

System: uname_result(system='Linux', node='ubuntu', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 4
Python 3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0]
numpy 2.2.6  scipy 1.15.2  h5py 3.13.0
Date: Thu Jun 26 23:10:18 2025
PySCF version 2.8.0
PySCF path  /home/whb/Downloads/miniconda3/envs/TDDFT/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C     -0.000002000000   0.525771000000   0.000000000000 AA   -0.000003779452   0.993563194239   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.977659000000   1.090978000000   0.000000000000 AA    1.847507753216   2.06164962792